# Mercedes-Benz Greener Manufacturing
Recent competitions are complicated and difficult for beginners.<br>
I'm a beginner, so I'm trying to do past competitions.<br>
It's a very simple code, but please comment if there are any improvements to this code! !!

<br>
Private Score 0.51903<br>
Public Score 0.53255<br>


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# import data

In [ ]:
sub = pd.read_csv("../input/mercedes-benz-greener-manufacturing/sample_submission.csv.zip")
test = pd.read_csv("../input/mercedes-benz-greener-manufacturing/test.csv.zip")
train = pd.read_csv("../input/mercedes-benz-greener-manufacturing/train.csv.zip")

In [ ]:
train["set"] = "train"
train.loc[(train['ID'] > 6000), 'set'] = 'va'
train

# check "y" destribution

In [ ]:
plt.style.use('default')
sns.set()
sns.set_style('whitegrid')
sns.set_palette('Dark2')

sns.distplot(train.y)

# make column named "set" to separate train, test and validation later

In [ ]:
test["set"] = "test"
test_id = test.ID

In [ ]:
data_all = pd.DataFrame()
data_all = pd.concat([train,test])
data_all = data_all.sort_values(by='ID', ascending=True)
#data_all = data_all.drop(columns = "ID")
data_all = data_all.reset_index(drop=True)
data_all

In [ ]:
enc = data_all[['X0','X1','X2','X3','X4','X5','X6','X8']]
enc

# LabelEncoding

In [ ]:
from sklearn.preprocessing import LabelEncoder #Library for LabelEncoding

for c in enc:
    le = LabelEncoder()
    le.fit(enc[c])
    enc[c] = le.transform(enc[c])
    
enc

# join and see the all data

In [ ]:
data_all = data_all.drop(['X0','X1','X2','X3','X4','X5','X6','X8'], axis=1)
data_all = data_all.join(enc)
data_all

# check the ratio of data

In [ ]:
n_tr = (data_all.set == 'train').sum()
n_va = (data_all.set == 'va').sum()
n_te = (data_all.set == 'test').sum()
n = [n_tr, n_va, n_te]
n

In [ ]:
label = ["train","va","test"]

plt.pie(n, labels=label)
plt.gca().add_artist(plt.Circle((0, 0), 0.65, color='white'))
plt.axis('equal')

# Split the data

In [ ]:
train_fit = pd.DataFrame()
train_fit = data_all.loc[data_all.set == "train"]

train_fit_x = train_fit.drop(columns = "y")
train_fit_x = train_fit_x.drop(columns = "set")
train_fit_y = train_fit.y


va_fit = pd.DataFrame()
va_fit = data_all.loc[data_all.set == "va"]

va_fit_x = va_fit.drop(columns = "y")
va_fit_x = va_fit_x.drop(columns = "set")
va_fit_y = va_fit.y


test_fit = pd.DataFrame()
test_fit = data_all.loc[data_all.set == "test"]

test_fit_x = test_fit.drop(['y', 'set'], axis=1)

# LightGBM

In [ ]:
%%time
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score

categorical_features = ['X0','X1','X2','X3','X4','X5','X6','X8']

# Parameter setting
params = {
    "objective" : "regression",
    "metric" : "rmse",
    "num_leaves" : 40,
    "learning_rate" : 0.01,
    "bagging_fraction" : 0.8,
    "feature_fraction" : 0.8, # Use only 〇% of features
    "bagging_frequency" : 6,
    "bagging_seed" : 42,
    "verbosity" : -1, # Whether to display the progress during learning
    "seed": 42
}

train_lgb = lgb.Dataset(train_fit_x,
                        label=train_fit_y,
                        categorical_feature=categorical_features)

va_lgb = lgb.Dataset(va_fit_x,
                     label=va_fit_y,
                     categorical_feature=categorical_features)

evals_result = {}

model_lgb = lgb.train(params, train_set = train_lgb,
                  num_boost_round = 10000,
                  valid_sets=[train_lgb, va_lgb], 
                  early_stopping_rounds=200, 
                  verbose_eval=1000,
                  evals_result=evals_result)

In [ ]:
va_y = va_fit_y

pred = model_lgb.predict(va_fit_x, num_iteration=model_lgb.best_iteration)

columns = ["y"]
pred_d = pd.DataFrame(data=pred, columns = columns)

# RSMLE score
score_RMSLE = np.sqrt(mean_squared_log_error(va_y, pred_d["y"]))
print("---------------RMSLE-score----------------")
print(score_RMSLE)

# RSMLE score
score_R2 = r2_score(va_y, pred_d["y"])
print("---------------R2-score----------------")
score_R2

# validation data visualization

In [ ]:
columns = ["y"]
va_fit_d = pd.DataFrame(data=va_y, columns = columns)
va_fit_d = va_fit_d.reset_index(drop=True)

sns.scatterplot(x=va_fit_d['y'], y=pred_d['y'])
sns.scatterplot(x=va_fit_d['y'], y=va_fit_d['y']) #perfect fitting line

# Predict and Submission

In [ ]:
# predict
pred = model_lgb.predict(test_fit_x)

pred_df = pd.DataFrame()
pred_df["y"] = pred
sns.distplot(pred_df.y)

In [ ]:
submission = pd.DataFrame()
submission["id"] = test_id
submission["y"] = pred

# Export to csv file
submission.to_csv("submission.csv", index=False)

In [ ]:
columns_ck = [["y"]]
submission_ck = pd.DataFrame(data=pred, columns = columns_ck)
test_id_df = pd.DataFrame(data=test_id)
test_id_df = test_id_df.reset_index(drop=True)
test_id_df.columns = ["id"]

test_reindex = test_fit_x.reset_index(drop=True)

Thank you for reading<br>
Please let me know anythin to imporve this code!!!<br>
## Also, I would be grateful if you could upvote to keep my motivation.
